In [114]:
import pandas as pd
from datetime import datetime
import re

In [132]:
df = pd.read_csv('../data/absorbance/raw.csv')

# extracting the year, so that we can remove the wrong data

In [133]:
regex = '^\w+.\w+,\d{2}/\d{2}/(\d{4})$'

In [134]:
df['year'] = df[df.columns.values[0]].apply(lambda x: re.findall(regex, x)[0])

# dropping data with wrong date

In [135]:
rows_to_drop = df[df['year'] == '1601']
df.drop(rows_to_drop.index, inplace=True)

In [136]:
df.reset_index(drop=True, inplace=True)

# removing unnecessary info

In [137]:
cols = list(df.columns.values[:2])
cols.append('year')

In [138]:
df.drop(cols, axis=1, inplace=True)

# saving new data

In [139]:
df.to_csv('../data/absorbance/treated.csv')